# Finite Temperature String Method

Umbrella sampling is an effective  tehcnique for sampling a well-defined path. In most cases, this means that the reaction coordinate can be expressed in one (or very few) dimension. However, in (bio)chemical systems, the processes  we are interested in are often much more complex and it can be challenging to find the stable structures and intermediates, let alone identify the optimal path connecting them.

In this section, we demonstrate the finite temperature string method, which aims to address these problems and iteratively optimises the pathway in a multidimensional space.

Let us get started with the necessary imports.

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline
sys.path.append("..")
from string_aux import *
from src.wham import WHAM

## Initial String on a Model potential

In this example, we use a two-dimensional analytical potential. There are two important things regarding the method, which you should remember:
- The number of biased dimensions/colvars can be arbitrarily high, and the _strings_ can be defined in any $X$-dimensional space.
- You may well be using a vast number of other degrees of freedom (for instance if the method is used with classical MD simulations). Selecting the colvars is highly contextual and an important scientific task on its own.

Let us say, we want to find the optimal path connecting the states in (-3,-2) and (3,2). One _string_ will consist of _windows_, similarly to umbrella sampling, and each window will have its biased position in every colvar.

We will sample every window with restraints in place. The strength of the restraints (here again harmonic, typical for distances) again has to be fit for the system, but in general it is *lower* than it would be in umbrella sampling.

In [ ]:
""" Parameters """
n_windows = 30  # Number of windows
n_steps = 1000  # Number of steps to run for
T = 300

restraint = np.empty((2, n_windows))    # distances in A
restraint[0] = np.linspace(-3, 3, n_windows)
restraint[1] = np.linspace(-1, 2, n_windows)
force = np.ones((2, n_windows)) * 2     # in kcal/(mol*A^2)

This will simply connect the two starting states by a straight line (in 2D), setting the windows equidistantly.

In [ ]:
plt.plot(restraint[0], restraint[1], "s-")
plt.xlabel("CV1 (Å)")
plt.ylabel("CV2 (Å)")
plt.show()

## Sampling in Windows

We will start each window from its centre. In this case we use a Monte-Carlo sampler, which drives the system towards lower energy. Here, it is important that the restraints will let each window's sampled population to deviate from the centre. Also, we will need to keep hold of the data we generate and the restraint positions.

In [ ]:
# set the starting positions
last_position = np.copy(restraint)
all_data = []
all_restraint = []
# first iteration
data, last_position = generate_data(n_windows, n_steps, T, force, last_position, restraint)
all_data.append(data)
all_restraint.append(restraint)

Take a look at the average position in the simulations. Also, we can plot the data for one of the windows to see where it's been.

In [ ]:
average_position = np.mean(data, axis=1)
plt.plot(restraint[0], restraint[1], "s-", label="Restraint position")
plt.plot(average_position[:, 0], average_position[:, 1], 'o', label="Average position")
window = np.random.randint(0, 29)
plt.plot(data[window, :, 0], data[window, :, 1], label="Window {:d} trajectory".format(window + 1),
        linewidth=0.2)
plt.legend()
plt.show()

## String optimisation

We see the simulations are grouping in different regions and tilt away from out ad-hoc initial string. For the next iteration, we create a new string, by fitting a polynomial to the averages and splitting it again equidistantly. (Other fits are also possible, sometimes beneficial)

In [ ]:
restraint = optimize_string(average_position)

plt.plot(all_restraint[0][0], all_restraint[0][1], "s-", label="String 1")
plt.plot(average_position[:, 0], average_position[:, 1], 'o', label="Average position")
plt.plot(restraint[0], restraint[1], "s-", label="String 2")
plt.legend()
plt.show()

Fast forward some iterations...

In [ ]:
niter = 15
for itr in range(niter):
    # Generate stochastic data
    data, last_position = generate_data(n_windows, n_steps, T, force, last_position, restraint)
    all_data.append(data)
    all_restraint.append(restraint)
    average_position = np.mean(data, axis=1)
    # Optimize the string for the next iteration
    restraint = optimize_string(average_position)

Now plot the strings we have so far.

In [ ]:
f, ax = plt.subplots()
color = cm.rainbow(np.linspace(0, 1, len(all_restraint)))
for i in range(len(all_restraint)):
    ax.plot(all_restraint[i][0], all_restraint[i][1], "-s", markersize=3, linewidth=1, color=color[i])
plt.show()

## Sampling for the Free Energy Profile

We can see the string starting to converge. On its own, knowing how the colvars change can lead us to some mechanistic insight of the process, but we can aspire for more. With enough sampling along the path, we can reconstruct the underlying potential using WHAM.

In [ ]:
niter = 15
for itr in range(niter):
    # Generate stochastic data
    data, last_position = generate_data(n_windows, n_steps, T, force, last_position, restraint)
    all_data.append(data)
    all_restraint.append(restraint)
    average_position = np.mean(data, axis=1)
    # Optimize the string for the next iteration
    restraint = optimize_string(average_position)

## Unbiasing

WHAM will be used similarly to the previous section, which needs some conversion the format.

In [ ]:
stride = 3      # may stride the data for saving, to save memory
all_data = np.concatenate(all_data, axis=0)[:, ::stride, :]
all_restraint = np.transpose(np.concatenate(all_restraint, axis=1))

We can skip some of the early strings, which will not contribute much to the profile, but will help us speed up.

In [ ]:
skip = 1
w = WHAM()
w.setup(all_data[int(skip * n_windows):], 300,
        np.transpose(np.concatenate([force for _ in range(int(len(all_data) / n_windows) - skip)], axis=1)),
        all_restraint[int(skip * n_windows):])
w.converge(0.01)


Now that we have a converged WHAM, we need to project into a discrete mesh. `[0, 1]` and `[1, 0]` selects the first and second colvars. In a case of a many-dimensional string, we may choose to define linear combinations. 30 is the number of bins.

In [ ]:
w.project_2d([[0, 1], [1, 0]], 30)

## Free Energy Profile

Finally, we can compare the profile with the true potential, access by the Epot function. We jump out to the Qt backend, so the figure can be viewed interactively. We also spice it up with the final string.

In [ ]:
%matplotlib qt
# prepare the last string
sx = all_restraint[-n_windows:, 0]
sy = all_restraint[-n_windows:, 1]
sz = Epot(sx, sy)
# we use the internal binning of WHAM
[x, y] = np.meshgrid(w.colvar1_bins, w.colvar2_bins)
fig = plt.figure(figsize=(20, 10))
# start with the true potential
ax_true = fig.add_subplot(121, projection='3d')
surf_true = ax_true.plot_surface(x, y, Epot(x, y), cmap=cm.viridis, vmin=0, vmax=8, zorder=1)
ax_true.plot(sx, sy, sz, "s-", markersize=4, linewidth=2, color="lime", zorder=3)
ax_true.set_zlim(0, 8)
ax_true.view_init(azim=0, elev=90)
# now the reconstruction
ax = fig.add_subplot(122, projection='3d')
surf = ax.plot_surface(x, y, w.profile2d, cmap=cm.viridis, vmin=0, vmax=8, zorder=1)
ax.plot(sx, sy, sz, "s-", markersize=4, linewidth=2, color="lime", zorder=3)
ax.set_zlim(0, 8)
ax.view_init(azim=0, elev=90)
plt.tight_layout()
plt.show()

## Optional Tasks

There are many parameters you can play with to observe different behaviour:
- Try different force constants: with higher, you will lose the overlap and cannot recover the profile.
- Number of windows: more windows are more expensive, less will again diminish the overlap of neighbours.
- Number of iterations, number of bins for projection, WHAM threshold etc.